# SK ChatBot with Templates

This will not use SK chat handling instead it will use Promptflow template concepts for handling chat state.## Load the required .NET packages and supporting classes

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta8"
#r "nuget: Scriban"
#r "nuget: dotenv.net"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;
using dotenv.net;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;
using Scriban;

## Load the OpenAI variables from environment variables or an .env file

In [ ]:
DotEnv.Load();
var deploymentName = Environment.GetEnvironmentVariable("GPT_OPENAI_DEPLOYMENT_NAME");
var endpoint = Environment.GetEnvironmentVariable("GPT_OPENAI_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("GPT_OPENAI_KEY");
Console.WriteLine($"Using deployment: {deploymentName} at {endpoint} with key {apiKey.Substring(0, 5)}...");

## Build a kernel instance

- Registration: `WithAzureChatCompletionServices` registers an ICompletion service with the kernel
- Retries: Kernel API calls come with default retry logic for timeouts and throttling.

In [ ]:
var kernel = new KernelBuilder()
    .WithAzureOpenAIChatCompletionService(deploymentName, endpoint, apiKey)
    .Build();

## Get the registered service from the kernel and keep a conversation (a history)

## Define the template

In [ ]:
var template = Template.Parse(@"
system:
You are a helpful assistant.
{{ for item in chat_history }}user:
{{ item.question }}
assistant:
{{ item.answer }}
{{ end }}user:
{{ question }}
");
string question = string.Empty;
string templateResult = string.Empty;


## Enter the while loop and process the user and system messages

In [ ]:
while (true) {
    // Get the questions
    question = await InteractiveKernel.GetInputAsync("Your message");
    Console.WriteLine($"User: {question}");
    // Render the template
    templateResult = template.Render(new { question=question, chat_history = chat_history });
    // Create a semantic function
    var chatfunc = kernel.CreateSemanticFunction(templateResult,new OpenAIRequestSettings() { MaxTokens = 200, Temperature = 0.3, TopP = 1 });
    // Execute the function
    var answer = await kernel.RunAsync(chatfunc);
    // Print the answer
    Console.WriteLine($"Bot: {answer}");
    // Save the answer to history
    chat_history.Add(new Message(answer.ToString(), question));
    // Keep only the last 10 messages
    if (chat_history.Count > 10) {
        chat_history.RemoveAt(0);
    }
}